This is a program which is in testing phase. We will use a linear regression ML model to give an SOC number for the flights.

In [1]:
# Testing flights
train_one = 4620
# train_two = 4929

# Training flight
test_one = 4929

In [2]:
# Import the querying module
from flight_querying import query_flights
import pandas as pd

# Set up and retrieve the data from the database.
db_connect = query_flights()

In [4]:
# train_one_data = db_connect.connect_flight_for_ml(train_one)
# train_two_data = db_connect.connect_flight_for_ml(train_two)

# Set up train data
# train_x = pd.concat([train_one_data, train_two_data], axis=0)
train_x = db_connect.connect_flight_for_ml_data_label(train_one)
train_y = train_x["soc"].to_numpy()
numeric_train_x = train_x.drop(columns=["exercise", "soc"])

# Set up test data
test_x = db_connect.connect_flight_for_ml_data_label(test_one)
test_y = test_x["soc"].to_numpy()
numeric_test_x = test_x.drop(columns=["exercise", "soc"])

In [5]:
print(f"Length train_x = {len(train_x)} \n Length train_y = {len(train_y)}")

Length train_x = 31306 
 Length train_y = 31306


In [6]:
print(f"Length test_x = {len(test_x)} \n Length test_y = {len(test_y)}")

Length test_x = 21749 
 Length test_y = 21749


In [7]:
train_ids = []
for id in train_x["id"]:
    if id not in train_ids:
        train_ids.append(id)

print(train_ids)

[4620]


In [8]:
ids = []
for id in test_x["id"]:
    if id not in ids:
        ids.append(id)

print(ids)

[4929]


One-Hot-Encoding of the Operations columns

In [9]:
# ONE-HOT ENCODE
# https://stackabuse.com/one-hot-encoding-in-python-with-pandas-and-scikit-learn/
def one_hot(df, col, pre):
  encoded = pd.get_dummies(df[col], prefix=pre)
  for column in encoded:
    encoded = encoded.rename(columns={column: col + "_" + column})
  encoded['time'] = df['time']
  encoded["id"] = df["id"]
  return encoded

In [11]:
# Encode Train data
train_encoded = one_hot(train_x, "exercise", 'is')
numeric_train_x = pd.merge(numeric_train_x, train_encoded, on=["time","id"])

# Encode Test data
test_encoded = one_hot(test_x, "exercise", 'is')
numeric_test_x = pd.merge(numeric_test_x, test_encoded, on=["time","id"])

In [12]:
print(f"Length test_encoded = {len(test_encoded)} \n Length train_encoded = {len(train_encoded)}")

Length test_encoded = 21749 
 Length train_encoded = 31306


In [13]:
print(f"Length train_x = {len(numeric_train_x)} \n Length train_y = {len(train_y)}")

Length train_x = 31306 
 Length train_y = 31306


In [14]:
print(f"Length test_x = {len(numeric_test_x)} \n Length test_y = {len(test_y)}")

Length test_x = 21749 
 Length test_y = 21749


In [15]:
train_encoded

,exercise_is_NA,exercise_is_climb,exercise_is_cruise,exercise_is_descent,exercise_is_landing,exercise_is_takeoff,time,id
0,True,False,False,False,False,False,0.000000,4620
1,True,False,False,False,False,False,0.035273,4620
2,True,False,False,False,False,False,0.037968,4620
3,True,False,False,False,False,False,0.038609,4620
4,True,False,False,False,False,False,0.041305,4620
...,...,...,...,...,...,...,...,...
31301,True,False,False,False,False,False,52.446475,4620
31302,True,False,False,False,False,False,52.448142,4620
31303,True,False,False,False,False,False,52.449810,4620
31304,True,False,False,False,False,False,52.450985,4620


In [16]:
test_encoded

,exercise_is_NA,exercise_is_climb,exercise_is_cruise,exercise_is_descent,exercise_is_landing,exercise_is_takeoff,time,id
0,True,False,False,False,False,False,0.026925,4929
1,True,False,False,False,False,False,0.028639,4929
2,True,False,False,False,False,False,0.030300,4929
3,True,False,False,False,False,False,0.031973,4929
4,True,False,False,False,False,False,0.033600,4929
...,...,...,...,...,...,...,...,...
21744,True,False,False,False,False,False,36.446423,4929
21745,True,False,False,False,False,False,36.449079,4929
21746,True,False,False,False,False,False,36.449757,4929
21747,True,False,False,False,False,False,36.452369,4929


Machine Learning Model Implementation

In [17]:
# import sklearn
from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression

# Set model
regression_model = LinearRegression()

# Fit model
regression_model.fit(numeric_train_x, train_y)

LinearRegression()

In [18]:
# print model score
print(regression_model.score(numeric_test_x, test_y))

-1.634985811308491


In [21]:
coeff = pd.DataFrame(list(zip(regression_model.feature_names_in_, regression_model.coef_)), columns = ['Feature', 'Weight'])
coeff.sort_values('Weight')

,Feature,Weight
9,outside_air_temperature,-2.332470e+00
2,cell_temperature,-2.089099e+00
14,exercise_is_climb,-2.063747e+00
0,time,-1.650028e+00
15,exercise_is_cruise,-1.384217e+00
11,pitch,-3.059662e-01
16,exercise_is_descent,-2.637390e-01
5,motor_temperature,-1.917642e-01
4,motor_power,-1.456021e-01
13,exercise_is_NA,-1.242933e-01
